## Pada kode ini, kami mencoba menggunakan pyspark melalui google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=1c73397921650b4754adf89b93423e3bcd77dc35a2a9b57a6854b9d911cb057f
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
df = spark.read.csv('/content/drive/MyDrive/Airplane_Crashes_and_Fatalities_Since_1908.csv', header=True)

In [ ]:
df.show(5)
df.count()

+----------+-----+--------------------+--------------------+--------+-------------+--------------------+------------+-----+------+----------+------+--------------------+
|      Date| Time|            Location|            Operator|Flight #|        Route|                Type|Registration|cn/In|Aboard|Fatalities|Ground|             Summary|
+----------+-----+--------------------+--------------------+--------+-------------+--------------------+------------+-----+------+----------+------+--------------------+
| 9/17/1908|17:18| Fort Myer, Virginia|Military - U.S. Army|    null|Demonstration|    Wright Flyer III|        null|    1|     2|         1|     0|During a demonstr...|
| 7/12/1912| 6:30|AtlantiCity, New ...|Military - U.S. Navy|    null|  Test flight|           Dirigible|        null| null|     5|         5|     0|First U.S. dirigi...|
|  8/6/1913| null|Victoria, British...|             Private|       -|         null|    Curtiss seaplane|        null| null|     1|         1|     0|Th

5268

## Data Cleansing

In [ ]:
df = df.na.drop(subset=['Summary'])

In [ ]:
df.select("Summary").show(10, truncate=False)
df.count()

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Summary                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

4878

In [ ]:
# Kata untuk dihitung sesuai cluster
cluster_words = [
    'aircraft', 'crashed', 'plane', 'shortly', 'taking', 'airport', 'altitude',
    'crew', 'due', 'engine', 'failed', 'failure', 'fire', 'flight', 'landing',
    'lost', 'pilot', 'runway', 'takeoff', 'fog', 'attempting', 'cargo', 'land',
    'miles', 'route', 'struck', 'accident', 'approach', 'conditions', 'control',
    'feet', 'flying', 'ground', 'killed', 'low', 'mountain', 'poor', 'short',
    'weather', 'left', 'loss'
]

## Menghitung kata sering muncul per baris sebagai kata kunci

In [ ]:
word_counts = {}

# Iterasi melalui setiap baris DataFrame
for row in df.collect():
    summary = row['Summary']
    words = summary.split(' ')
    
    # Hitung jumlah kata dalam cluster_words
    word_count = {}
    for word in words:
        if word.lower() in cluster_words:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1
    
    # Menambahkan hasil perhitungan per baris ke word_counts
    word_counts[row['Summary']] = word_count

# Tampilkan hasil perhitungan per baris
for summary, counts in word_counts.items():
    print(f'Summary: {summary}')
    for word, count in counts.items():
        print(f'{word}: {count}')
    print()


Streaming output truncated to the last 5000 lines.
runway: 1

Summary: The aircraft crashed into the jungle after the rear engine was hit by a Russian  SA-7 missile, fired by Tutsi rebel forces shortly after the plane took off from Kindu Airport bound for Kinshasa.
aircraft: 1
crashed: 1
engine: 1
shortly: 1
plane: 1
Airport: 1

Summary: While attempting to take off, the aircraft struck a military bus as it crossed the runway.
attempting: 1
aircraft: 1
struck: 1

Summary: The cargo was on approach when it struck electrical cables and crashed into a house.
cargo: 1
approach: 1
struck: 1
crashed: 1

Summary: Crashed in the Rwenzori mountains.
Crashed: 1

Summary: The aircraft crashed into mountainous terrain in fog and rain 44 minutes after leaving Playa Grande, Guatemala. The plane was returning from doing a medical clinifor 3 days in the remote jungles of Playa Grande where no one had seen a doctor before.
aircraft: 1
crashed: 1
fog: 1
plane: 1

Summary: The aircraft, carrying freight 

In [1]:
# clusters = {
#     1: ["aircraft", "crashed", "plane", "shortly", "taking"],
#     2: ["aircraft", "airport", "altitude", "crashed", "crew", "due", "engine", "failed", "failure", "fire", "flight", "landing", "lost", "pilot", "plane", "runway", "takeoff", "taking"],
#     3: ["aircraft", "crashed", "fog"],
#     4: ["aircraft", "airport", "attempting", "cargo", "crashed", "fire", "land", "landing", "miles", "pilot", "plane", "route", "runway", "struck", "takeoff"],
#     5: ["accident", "aircraft", "airport", "altitude", "approach", "attempting", "cargo", "conditions", "control", "crashed", "crew", "due", "engine", "failed", "failure", "feet", "fire", "flight", "flying", "fog", "ground", "killed", "land", "landing", "lost", "low", "miles", "mountain", "pilot", "plane", "poor", "route", "runway", "short", "shortly", "struck", "takeoff", "taking", "weather"],
#     6: ["aircraft", "crashed", "engine", "failure", "fire", "flight", "left", "pilot", "plane", "runway"],
#     7: ["accident", "aircraft", "altitude", "cargo", "control", "crashed", "crew", "due", "engine", "failure", "flight", "landing", "loss", "lost", "pilot", "plane", "takeoff"]
# }

# cluster_counts = {cluster: 0 for cluster in clusters}

# # Iterasi melalui setiap baris DataFrame
# for row in df.collect():
#     summary = row['Summary']
#     words = summary.split(' ')
    
#     cluster_scores = {cluster: 0 for cluster in clusters}
    
#     # Hitung skor setiap cluster berdasarkan jumlah kemunculan kata dalam cluster_words
#     for word in words:
#         for cluster, words_list in clusters.items():
#             if word.lower() in words_list:
#                 cluster_scores[cluster] += 1
    
#     # Tentukan cluster dengan skor tertinggi
#     max_score = max(cluster_scores.values())
#     max_clusters = [cluster for cluster, score in cluster_scores.items() if score == max_score]
    
#     # Tampilkan hasil perbandingan nilai setiap cluster dan jumlah kemunculan kata yang dimaksud
#     print(f'Summary: {summary}')
#     for cluster in clusters:
#         cluster_score = cluster_scores[cluster]
#         words_count = sum(1 for word in words if word.lower() in clusters[cluster])
#         print(f'Cluster {cluster}: Score = {cluster_score}, Kata: {words_count}')
#     print(f'Masuk Cluster: {max_clusters}\n')
#     cluster_counts[max_clusters[0]] += 1

#     # Jika baris masuk ke lebih dari satu cluster
#     if len(max_clusters) > 1:
#         # Anda dapat menentukan metode pengambilan keputusan, misalnya berdasarkan persentase kemunculan kata yang dimaksud dalam masing-masing cluster
#         # Misalnya, jika 2 cluster memiliki kemunculan kata yang sama, Anda dapat memilih kluster dengan persentase kemunculan kata yang lebih tinggi
#         cluster_percentages = {cluster: cluster_scores[cluster] / len(words) for cluster in max_clusters}
#         selected_cluster = max(cluster_percentages, key=cluster_percentages.get)
#     else:
#         selected_cluster = max_clusters[0]
    
#     cluster_counts[selected_cluster] += 1

# # Tampilkan perbandingan jumlah baris yang masuk ke setiap cluster
# print('Perbandingan Jumlah Baris per Cluster:')
# for cluster, count in cluster_counts.items():
#     print(f'Cluster {cluster}: {count} baris')
